In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import os

In [ ]:
# Dataset containing ESRB ratings for Switch, Xbox One, and PS4 games

df = pd.read_pickle('esrb_ratings.pkl')

df.head()

In [ ]:
# Drop any characters from game titles that might interfere

titles = []
bad_chars = ['*', ':', ';', '+', '~', '-', '/', '.', '?', '!',
             '<', '>', '‘', "'", '"', '™', '®']
for title in df['title']:
    for char in bad_chars:
        title = title.replace(char, '')
    titles.append(title)

titles[:10]

In [ ]:
# Replace titles in dataframe with edited titles

df['title'] = titles

df['title'][:10]

In [ ]:
df[:10]

In [ ]:
# Make a directory for each rating

for x in df['rating'].unique():
    os.mkdir('./images/' + x)

In [ ]:
# Start webscraper

option = webdriver.ChromeOptions()
option.add_argument('incognito')

driver = webdriver.Chrome(options=option)

In [ ]:
# Go to Google Images
driver.get('https://google.com/images')

filenames = []
for idx in range(len(df)):
    try:
        # Search each title
        driver.find_element(By.NAME, 'q').clear()
        driver.find_element(By.NAME, 'q').send_keys(df['title'][idx] + ' video game screenshot' + Keys.RETURN)
        
        # Grab the first ten images
        images = driver.find_elements_by_class_name('rg_i')
        print('#' + str(idx) + ' --- ' + df['title'][idx])
        for i, image in enumerate(images[:10]):
            filename = './images/' + df['rating'][idx] + '/' + df['title'][idx] + '_' + str(i) + '.png'
            with open(filename, 'wb') as file:
                file.write(image.screenshot_as_png)
            
            filenames.append(filename)
        print('Done!')
        
        # Wait so Google doesn't freak out
        time.sleep(3)
    except:
        print('#' + str(idx) + '---' + df['title'][idx])
        print('ERROR')
        continue